# 한 문장으로 정의:
입력1: 한 사람의 2D영상(.mp4) - youtube에서 다운로드

출력1: 2D영상의 keypoint(.json, mp4) - Using AlphaPose
 https://github.com/shshjhjh4455/MotionBERT/blob/main/sample/AlphaPose_4.mp4, https://github.com/shshjhjh4455/MotionBERT/blob/main/sample/alphapose-results.json

입력2: 출력1

출력2: 3D영상([3D Pose](https://github.com/shshjhjh4455/MotionBERT/blob/main/infer_wild.py), [Mesh]((https://github.com/shshjhjh4455/MotionBERT/blob/main/infer_wild_mesh.py))) (.mp4) - https://github.com/shshjhjh4455/MotionBERT/tree/main/sample/output 출력소요시간 약40분 using colab+ A100 GPU 





# 해당 프로젝트의 핵심 알고리즘 -> DSTfromer

## 소개:
-DSTformer은 3D 자세 추정과 동작 인식 분야에서 중요한 역할을 하는 딥러닝 모델입니다.

-자세 추정과 동작 인식은 인간의 자세와 동작을 정확하게 추정하고 분류하는 작업으로, 로봇 제어, 운동 분석, 가상 현실 등 다양한 응용 분야에서 활용됩니다.

-DSTformer은 시간(Temporal)과 공간(Spatial) 정보를 효과적으로 모델링하여 입력 데이터의 시공간적인 패턴을 학습하고, 시간과 공간의 상호작용을 포착할 수 있는 딥러닝 기반 알고리즘입니다.

-이를 통해 DSTformer는 정확한 3D 자세 추정과 동작 인식을 수행할 수 있으며, 이는 인간의 움직임을 이해하고 상황에 맞는 응답을 제공하는 다양한 응용 분야에서 큰 가치를 가지게 됩니다.

## 알고리즘 개요:
[Feature1](https://github.com/shshjhjh4455/MotionBERT/blob/main/img/Screenshot%202023-06-13%20at%2012.29.05%20PM.png)

[Feature2](https://github.com/shshjhjh4455/MotionBERT/blob/main/img/Screenshot%202023-06-13%20at%2012.29.24%20PM.png)

DSTformer 알고리즘의 핵심 아이디어를 간략히 설명합니다.

시간과 공간 정보를 효과적으로 모델링하기 위해 Dual Spatial-Temporal Transformer를 사용한다는 점을 강조합니다.

예시: "DSTformer 알고리즘은 시간(Temporal)과 공간(Spatial) 정보를 효과적으로 모델링하기 위해 Dual Spatial-Temporal Transformer를 사용합니다. 이를 통해 DSTformer는 입력 데이터의 시간적 및 공간적인 패턴을 모델링하고, 시간과 공간의 상호작용을 포착할 수 있습니다."

### 발표는 해당 프로젝트의 핵심 알고리즘인 DSTformer에 대해서 설명합니다. DSTformer.py는 [learning.py](https://github.com/shshjhjh4455/MotionBERT/blob/main/lib/utils/learning.py)에서 사용되며, 이는 [학습](https://github.com/shshjhjh4455/MotionBERT/blob/main/docs/action.md) 시 사용됩니다. 

## 네트워크 구조:
DSTformer의 전체적인 네트워크 구조를 설명합니다.

MLP, Attention, Block 등의 구성 요소에 대한 역할과 동작 방식을 설명합니다.

예시: "DSTformer는 MLP, Attention, Block 등의 구성 요소로 이루어진 네트워크 구조로 구성됩니다. MLP는 다층 퍼셉트론으로, 입력 데이터에 대한 선형 변환과 활성화 함수(GELU)를 적용하고 일부 뉴런을 무작위로 비활성화(Dropout)합니다. Attention은 입력 데이터의 시간적 및 공간적인 상호작용을 계산하고 중요한 정보에 가중치를 부여합니다. Block은 여러 개의 MLP와 Attention 레이어로 구성되어 시간 및 공간적 관계를 모델링합니다."

#### '원본 코드 설명' + '수정 및 개선한 부분'

In [ ]:
# 원본코드
import torch
import torch.nn as nn
import math
import warnings
import random
import numpy as np
from collections import OrderedDict
from functools import partial
from itertools import repeat
from lib.model.drop import DropPath

def _no_grad_trunc_normal_(tensor, mean, std, a, b):
    # Cut & paste from PyTorch official master until it's in a few official releases - RW
    # Method based on https://people.sc.fsu.edu/~jburkardt/presentations/truncated_normal.pdf
    def norm_cdf(x):
        # Computes standard normal cumulative distribution function
        return (1. + math.erf(x / math.sqrt(2.))) / 2.

    if (mean < a - 2 * std) or (mean > b + 2 * std):
        warnings.warn("mean is more than 2 std from [a, b] in nn.init.trunc_normal_. "
                      "The distribution of values may be incorrect.",
                      stacklevel=2)

    with torch.no_grad():
        # Values are generated by using a truncated uniform distribution and
        # then using the inverse CDF for the normal distribution.
        # Get upper and lower cdf values
        l = norm_cdf((a - mean) / std)
        u = norm_cdf((b - mean) / std)

        # Uniformly fill tensor with values from [l, u], then translate to
        # [2l-1, 2u-1].
        tensor.uniform_(2 * l - 1, 2 * u - 1)

        # Use inverse cdf transform for normal distribution to get truncated
        # standard normal
        tensor.erfinv_()

        # Transform to proper mean, std
        tensor.mul_(std * math.sqrt(2.))
        tensor.add_(mean)

        # Clamp to ensure it's in the proper range
        tensor.clamp_(min=a, max=b)
        return tensor


def trunc_normal_(tensor, mean=0., std=1., a=-2., b=2.):
    # type: (Tensor, float, float, float, float) -> Tensor
    r"""Fills the input Tensor with values drawn from a truncated
    normal distribution. The values are effectively drawn from the
    normal distribution :math:`\mathcal{N}(\text{mean}, \text{std}^2)`
    with values outside :math:`[a, b]` redrawn until they are within
    the bounds. The method used for generating the random values works
    best when :math:`a \leq \text{mean} \leq b`.
    Args:
        tensor: an n-dimensional `torch.Tensor`
        mean: the mean of the normal distribution
        std: the standard deviation of the normal distribution
        a: the minimum cutoff value
        b: the maximum cutoff value
    Examples:
        >>> w = torch.empty(3, 5)
        >>> nn.init.trunc_normal_(w)
    """
    return _no_grad_trunc_normal_(tensor, mean, std, a, b)


class MLP(nn.Module):
    def __init__(self, in_features, hidden_features=None, out_features=None, act_layer=nn.GELU, drop=0.):
        super().__init__()
        out_features = out_features or in_features
        hidden_features = hidden_features or in_features
        self.fc1 = nn.Linear(in_features, hidden_features)
        self.act = act_layer()
        self.fc2 = nn.Linear(hidden_features, out_features)
        self.drop = nn.Dropout(drop)

    def forward(self, x):
        x = self.fc1(x)
        x = self.act(x)
        x = self.drop(x)
        x = self.fc2(x)
        x = self.drop(x)
        return x


class Attention(nn.Module):
    def __init__(self, dim, num_heads=8, qkv_bias=False, qk_scale=None, attn_drop=0., proj_drop=0., st_mode='vanilla'):
        super().__init__()
        self.num_heads = num_heads
        head_dim = dim // num_heads
        # NOTE scale factor was wrong in my original version, can set manually to be compat with prev weights
        self.scale = qk_scale or head_dim ** -0.5

        self.attn_drop = nn.Dropout(attn_drop)
        self.proj = nn.Linear(dim, dim)
        self.mode = st_mode
        if self.mode == 'parallel':
            self.ts_attn = nn.Linear(dim*2, dim*2)
            self.qkv = nn.Linear(dim, dim * 3, bias=qkv_bias)
        else:
            self.qkv = nn.Linear(dim, dim * 3, bias=qkv_bias)
        self.proj_drop = nn.Dropout(proj_drop)

        self.attn_count_s = None
        self.attn_count_t = None

    def forward(self, x, seqlen=1):
        B, N, C = x.shape
        
        if self.mode == 'series':
            qkv = self.qkv(x).reshape(B, N, 3, self.num_heads, C // self.num_heads).permute(2, 0, 3, 1, 4)
            q, k, v = qkv[0], qkv[1], qkv[2]   # make torchscript happy (cannot use tensor as tuple)
            x = self.forward_spatial(q, k, v)
            qkv = self.qkv(x).reshape(B, N, 3, self.num_heads, C // self.num_heads).permute(2, 0, 3, 1, 4)
            q, k, v = qkv[0], qkv[1], qkv[2]   # make torchscript happy (cannot use tensor as tuple)
            x = self.forward_temporal(q, k, v, seqlen=seqlen)
        elif self.mode == 'parallel':
            qkv = self.qkv(x).reshape(B, N, 3, self.num_heads, C // self.num_heads).permute(2, 0, 3, 1, 4)
            q, k, v = qkv[0], qkv[1], qkv[2]   # make torchscript happy (cannot use tensor as tuple)
            x_t = self.forward_temporal(q, k, v, seqlen=seqlen)
            x_s = self.forward_spatial(q, k, v)
            
            alpha = torch.cat([x_s, x_t], dim=-1)
            alpha = alpha.mean(dim=1, keepdim=True)
            alpha = self.ts_attn(alpha).reshape(B, 1, C, 2)
            alpha = alpha.softmax(dim=-1)
            x = x_t * alpha[:,:,:,1] + x_s * alpha[:,:,:,0]
        elif self.mode == 'coupling':
            qkv = self.qkv(x).reshape(B, N, 3, self.num_heads, C // self.num_heads).permute(2, 0, 3, 1, 4)
            q, k, v = qkv[0], qkv[1], qkv[2]   # make torchscript happy (cannot use tensor as tuple)
            x = self.forward_coupling(q, k, v, seqlen=seqlen)
        elif self.mode == 'vanilla':
            qkv = self.qkv(x).reshape(B, N, 3, self.num_heads, C // self.num_heads).permute(2, 0, 3, 1, 4)
            q, k, v = qkv[0], qkv[1], qkv[2]   # make torchscript happy (cannot use tensor as tuple)
            x = self.forward_spatial(q, k, v)
        elif self.mode == 'temporal':
            qkv = self.qkv(x).reshape(B, N, 3, self.num_heads, C // self.num_heads).permute(2, 0, 3, 1, 4)
            q, k, v = qkv[0], qkv[1], qkv[2]   # make torchscript happy (cannot use tensor as tuple)
            x = self.forward_temporal(q, k, v, seqlen=seqlen)
        elif self.mode == 'spatial':
            qkv = self.qkv(x).reshape(B, N, 3, self.num_heads, C // self.num_heads).permute(2, 0, 3, 1, 4)
            q, k, v = qkv[0], qkv[1], qkv[2]   # make torchscript happy (cannot use tensor as tuple)
            x = self.forward_spatial(q, k, v)
        else:
            raise NotImplementedError(self.mode)
        x = self.proj(x)
        x = self.proj_drop(x)
        return x
    
    def reshape_T(self, x, seqlen=1, inverse=False):
        if not inverse:
            N, C = x.shape[-2:]
            x = x.reshape(-1, seqlen, self.num_heads, N, C).transpose(1,2)
            x = x.reshape(-1, self.num_heads, seqlen*N, C) #(B, H, TN, c)
        else:
            TN, C = x.shape[-2:]
            x = x.reshape(-1, self.num_heads, seqlen, TN // seqlen, C).transpose(1,2)
            x = x.reshape(-1, self.num_heads, TN // seqlen, C) #(BT, H, N, C)
        return x 

    def forward_coupling(self, q, k, v, seqlen=8):
        BT, _, N, C = q.shape
        q = self.reshape_T(q, seqlen)
        k = self.reshape_T(k, seqlen)
        v = self.reshape_T(v, seqlen)

        attn = (q @ k.transpose(-2, -1)) * self.scale
        attn = attn.softmax(dim=-1)
        attn = self.attn_drop(attn)

        x = attn @ v
        x = self.reshape_T(x, seqlen, inverse=True)
        x = x.transpose(1,2).reshape(BT, N, C*self.num_heads)
        return x

    def forward_spatial(self, q, k, v):
        B, _, N, C = q.shape
        attn = (q @ k.transpose(-2, -1)) * self.scale
        attn = attn.softmax(dim=-1)
        attn = self.attn_drop(attn)

        x = attn @ v
        x = x.transpose(1,2).reshape(B, N, C*self.num_heads)
        return x
        
    def forward_temporal(self, q, k, v, seqlen=8):
        B, _, N, C = q.shape
        qt = q.reshape(-1, seqlen, self.num_heads, N, C).permute(0, 2, 3, 1, 4) #(B, H, N, T, C)
        kt = k.reshape(-1, seqlen, self.num_heads, N, C).permute(0, 2, 3, 1, 4) #(B, H, N, T, C)
        vt = v.reshape(-1, seqlen, self.num_heads, N, C).permute(0, 2, 3, 1, 4) #(B, H, N, T, C)

        attn = (qt @ kt.transpose(-2, -1)) * self.scale
        attn = attn.softmax(dim=-1)
        attn = self.attn_drop(attn)

        x = attn @ vt #(B, H, N, T, C)
        x = x.permute(0, 3, 2, 1, 4).reshape(B, N, C*self.num_heads)
        return x

    def count_attn(self, attn):
        attn = attn.detach().cpu().numpy()
        attn = attn.mean(axis=1)
        attn_t = attn[:, :, 1].mean(axis=1)
        attn_s = attn[:, :, 0].mean(axis=1)
        if self.attn_count_s is None:
            self.attn_count_s = attn_s
            self.attn_count_t = attn_t
        else:
            self.attn_count_s = np.concatenate([self.attn_count_s, attn_s], axis=0)
            self.attn_count_t = np.concatenate([self.attn_count_t, attn_t], axis=0)

class Block(nn.Module):

    def __init__(self, dim, num_heads, mlp_ratio=4., mlp_out_ratio=1., qkv_bias=True, qk_scale=None, drop=0., attn_drop=0.,
                 drop_path=0., act_layer=nn.GELU, norm_layer=nn.LayerNorm, st_mode='stage_st', att_fuse=False):
        super().__init__()
        # assert 'stage' in st_mode
        self.st_mode = st_mode
        self.norm1_s = norm_layer(dim)
        self.norm1_t = norm_layer(dim)
        self.attn_s = Attention(
            dim, num_heads=num_heads, qkv_bias=qkv_bias, qk_scale=qk_scale, attn_drop=attn_drop, proj_drop=drop, st_mode="spatial")
        self.attn_t = Attention(
            dim, num_heads=num_heads, qkv_bias=qkv_bias, qk_scale=qk_scale, attn_drop=attn_drop, proj_drop=drop, st_mode="temporal")
        
        # NOTE: drop path for stochastic depth, we shall see if this is better than dropout here
        self.drop_path = DropPath(drop_path) if drop_path > 0. else nn.Identity()
        self.norm2_s = norm_layer(dim)
        self.norm2_t = norm_layer(dim)
        mlp_hidden_dim = int(dim * mlp_ratio)
        mlp_out_dim = int(dim * mlp_out_ratio)
        self.mlp_s = MLP(in_features=dim, hidden_features=mlp_hidden_dim, out_features=mlp_out_dim, act_layer=act_layer, drop=drop)
        self.mlp_t = MLP(in_features=dim, hidden_features=mlp_hidden_dim, out_features=mlp_out_dim, act_layer=act_layer, drop=drop)
        self.att_fuse = att_fuse
        if self.att_fuse:
            self.ts_attn = nn.Linear(dim*2, dim*2)
    def forward(self, x, seqlen=1):
        if self.st_mode=='stage_st':
            x = x + self.drop_path(self.attn_s(self.norm1_s(x), seqlen))
            x = x + self.drop_path(self.mlp_s(self.norm2_s(x)))
            x = x + self.drop_path(self.attn_t(self.norm1_t(x), seqlen))
            x = x + self.drop_path(self.mlp_t(self.norm2_t(x)))
        elif self.st_mode=='stage_ts':
            x = x + self.drop_path(self.attn_t(self.norm1_t(x), seqlen))
            x = x + self.drop_path(self.mlp_t(self.norm2_t(x)))
            x = x + self.drop_path(self.attn_s(self.norm1_s(x), seqlen))
            x = x + self.drop_path(self.mlp_s(self.norm2_s(x)))
        elif self.st_mode=='stage_para':
            x_t = x + self.drop_path(self.attn_t(self.norm1_t(x), seqlen))
            x_t = x_t + self.drop_path(self.mlp_t(self.norm2_t(x_t)))
            x_s = x + self.drop_path(self.attn_s(self.norm1_s(x), seqlen))
            x_s = x_s + self.drop_path(self.mlp_s(self.norm2_s(x_s)))
            if self.att_fuse:
                #             x_s, x_t: [BF, J, dim]
                alpha = torch.cat([x_s, x_t], dim=-1)
                BF, J = alpha.shape[:2]
                # alpha = alpha.mean(dim=1, keepdim=True)
                alpha = self.ts_attn(alpha).reshape(BF, J, -1, 2)
                alpha = alpha.softmax(dim=-1)
                x = x_t * alpha[:,:,:,1] + x_s * alpha[:,:,:,0]
            else:
                x = (x_s + x_t)*0.5
        else:
            raise NotImplementedError(self.st_mode)
        return x
    
class DSTformer(nn.Module):
    def __init__(self, dim_in=3, dim_out=3, dim_feat=256, dim_rep=512,
                 depth=5, num_heads=8, mlp_ratio=4, 
                 num_joints=17, maxlen=243, 
                 qkv_bias=True, qk_scale=None, drop_rate=0., attn_drop_rate=0., drop_path_rate=0., norm_layer=nn.LayerNorm, att_fuse=True):
        super().__init__()
        self.dim_out = dim_out
        self.dim_feat = dim_feat
        self.joints_embed = nn.Linear(dim_in, dim_feat)
        self.pos_drop = nn.Dropout(p=drop_rate)
        dpr = [x.item() for x in torch.linspace(0, drop_path_rate, depth)]  # stochastic depth decay rule
        self.blocks_st = nn.ModuleList([
            Block(
                dim=dim_feat, num_heads=num_heads, mlp_ratio=mlp_ratio, qkv_bias=qkv_bias, qk_scale=qk_scale,
                drop=drop_rate, attn_drop=attn_drop_rate, drop_path=dpr[i], norm_layer=norm_layer, 
                st_mode="stage_st")
            for i in range(depth)])
        self.blocks_ts = nn.ModuleList([
            Block(
                dim=dim_feat, num_heads=num_heads, mlp_ratio=mlp_ratio, qkv_bias=qkv_bias, qk_scale=qk_scale,
                drop=drop_rate, attn_drop=attn_drop_rate, drop_path=dpr[i], norm_layer=norm_layer, 
                st_mode="stage_ts")
            for i in range(depth)])
        self.norm = norm_layer(dim_feat)
        if dim_rep:
            self.pre_logits = nn.Sequential(OrderedDict([
                ('fc', nn.Linear(dim_feat, dim_rep)),
                ('act', nn.Tanh())
            ]))
        else:
            self.pre_logits = nn.Identity()
        self.head = nn.Linear(dim_rep, dim_out) if dim_out > 0 else nn.Identity()            
        self.temp_embed = nn.Parameter(torch.zeros(1, maxlen, 1, dim_feat))
        self.pos_embed = nn.Parameter(torch.zeros(1, num_joints, dim_feat))
        trunc_normal_(self.temp_embed, std=.02)
        trunc_normal_(self.pos_embed, std=.02)
        self.apply(self._init_weights)
        self.att_fuse = att_fuse
        if self.att_fuse:
            self.ts_attn = nn.ModuleList([nn.Linear(dim_feat*2, 2) for i in range(depth)])
            for i in range(depth):
                self.ts_attn[i].weight.data.fill_(0)
                self.ts_attn[i].bias.data.fill_(0.5)

    def _init_weights(self, m):
        if isinstance(m, nn.Linear):
            trunc_normal_(m.weight, std=.02)
            if isinstance(m, nn.Linear) and m.bias is not None:
                nn.init.constant_(m.bias, 0)
        elif isinstance(m, nn.LayerNorm):
            nn.init.constant_(m.bias, 0)
            nn.init.constant_(m.weight, 1.0)

    def get_classifier(self):
        return self.head

    def reset_classifier(self, dim_out, global_pool=''):
        self.dim_out = dim_out
        self.head = nn.Linear(self.dim_feat, dim_out) if dim_out > 0 else nn.Identity()

    def forward(self, x, return_rep=False):   
        B, F, J, C = x.shape
        x = x.reshape(-1, J, C)
        BF = x.shape[0]
        x = self.joints_embed(x)
        x = x + self.pos_embed
        _, J, C = x.shape
        x = x.reshape(-1, F, J, C) + self.temp_embed[:,:F,:,:]
        x = x.reshape(BF, J, C)
        x = self.pos_drop(x)
        alphas = []
        for idx, (blk_st, blk_ts) in enumerate(zip(self.blocks_st, self.blocks_ts)):
            x_st = blk_st(x, F)
            x_ts = blk_ts(x, F)
            if self.att_fuse:
                att = self.ts_attn[idx]
                alpha = torch.cat([x_st, x_ts], dim=-1)
                BF, J = alpha.shape[:2]
                alpha = att(alpha)
                alpha = alpha.softmax(dim=-1)
                x = x_st * alpha[:,:,0:1] + x_ts * alpha[:,:,1:2]
            else:
                x = (x_st + x_ts)*0.5
        x = self.norm(x)
        x = x.reshape(B, F, J, -1)
        x = self.pre_logits(x)         # [B, F, J, dim_feat]
        if return_rep:
            return x
        x = self.head(x)
        return x

    def get_representation(self, x):
        return self.forward(x, return_rep=True)
    

## 알고리즘 설명

### MLP:
MLP의 역할과 구조를 설명합니다.

입력 데이터에 대한 선형 변환과 활성화 함수(GELU)를 적용하고, 일부 뉴런을 무작위로 비활성화(Dropout)합니다.

예시: "MLP는 입력 데이터에 대한 선형 변환과 활성화 함수(GELU)를 적용하는 다층 퍼셉트론입니다. 이를 통해 입력 데이터의 비선형성을 모델링하고 일부 뉴런을 무작위로 비활성화하여 과적합을 방지합니다."

In [ ]:

class MLP(nn.Module):
    def __init__(self, in_features, hidden_features=None, out_features=None, act_layer=nn.GELU, drop=0.):
        super().__init__()
        out_features = out_features or in_features
        hidden_features = hidden_features or in_features
        self.fc1 = nn.Linear(in_features, hidden_features)
        self.act = act_layer()
        self.fc2 = nn.Linear(hidden_features, out_features)
        self.drop = nn.Dropout(drop)

    def forward(self, x):
        x = self.fc1(x)
        x = self.act(x)
        x = self.drop(x)
        x = self.fc2(x)
        x = self.drop(x)
        return x

          Input
            │
       ┌────▼────┐
  fc1 │          │
       ▼          ▼
   ┌────────┐  ┌────────┐
   │ Linear │  │ Linear │
   └────────┘  └────────┘
       │          │
    act()        act()
       │          │
   ┌────────┐  ┌────────┐
   │ Dropout│  │ Dropout│
   └────────┘  └────────┘
       │          │
   ┌────────┐  ┌────────┐
   │ Linear │  │ Linear │
   └────────┘  └────────┘
       │          │
      Output

위의 표현은 MLP 클래스의 구성 요소와 데이터 흐름을 나타내며, 순차적으로 진행됩니다.

입력 데이터가 fc1으로 들어가고, Linear 변환을 수행합니다.
act() 함수를 통해 활성화 함수가 적용됩니다.
Dropout을 통해 일부 뉴런이 무작위로 비활성화됩니다.
다시 Linear 변환을 수행하고, 다시 Dropout이 적용됩니다.
최종적으로 출력 데이터가 생성됩니다.
이를 통해 MLP 클래스는 입력 데이터를 변환하고 비선형성을 모델링하는 다층 퍼셉트론 구조를 나타냅니다.

### Attention:
Attention의 역할과 동작 방식을 설명합니다.

입력 데이터의 시간적 및 공간적인 상호작용을 계산하고, 중요한 정보에 가중치를 부여합니다.

예시: "Attention은 입력 데이터의 시간적 및 공간적인 상호작용을 계산하는 메커니즘입니다. 입력 데이터의 특성을 기반으로 유사도를 계산하고, 중요한 정보에 대한 가중치를 부여합니다. 이를 통해 모델은 입력 데이터에서 핵심적인 패턴을 추출할 수 있습니다."

In [ ]:

class Attention(nn.Module):
    def __init__(self, dim, num_heads=8, qkv_bias=False, qk_scale=None, attn_drop=0., proj_drop=0., st_mode='vanilla'):
        super().__init__()
        self.num_heads = num_heads
        head_dim = dim // num_heads
        # NOTE scale factor was wrong in my original version, can set manually to be compat with prev weights
        self.scale = qk_scale or head_dim ** -0.5

        self.attn_drop = nn.Dropout(attn_drop)
        self.proj = nn.Linear(dim, dim)
        self.mode = st_mode
        if self.mode == 'parallel':
            self.ts_attn = nn.Linear(dim*2, dim*2)
            self.qkv = nn.Linear(dim, dim * 3, bias=qkv_bias)
        else:
            self.qkv = nn.Linear(dim, dim * 3, bias=qkv_bias)
        self.proj_drop = nn.Dropout(proj_drop)

        self.attn_count_s = None
        self.attn_count_t = None

    def forward(self, x, seqlen=1):
        B, N, C = x.shape
        
        if self.mode == 'series':
            qkv = self.qkv(x).reshape(B, N, 3, self.num_heads, C // self.num_heads).permute(2, 0, 3, 1, 4)
            q, k, v = qkv[0], qkv[1], qkv[2]   # make torchscript happy (cannot use tensor as tuple)
            x = self.forward_spatial(q, k, v)
            qkv = self.qkv(x).reshape(B, N, 3, self.num_heads, C // self.num_heads).permute(2, 0, 3, 1, 4)
            q, k, v = qkv[0], qkv[1], qkv[2]   # make torchscript happy (cannot use tensor as tuple)
            x = self.forward_temporal(q, k, v, seqlen=seqlen)
        elif self.mode == 'parallel':
            qkv = self.qkv(x).reshape(B, N, 3, self.num_heads, C // self.num_heads).permute(2, 0, 3, 1, 4)
            q, k, v = qkv[0], qkv[1], qkv[2]   # make torchscript happy (cannot use tensor as tuple)
            x_t = self.forward_temporal(q, k, v, seqlen=seqlen)
            x_s = self.forward_spatial(q, k, v)
            
            alpha = torch.cat([x_s, x_t], dim=-1)
            alpha = alpha.mean(dim=1, keepdim=True)
            alpha = self.ts_attn(alpha).reshape(B, 1, C, 2)
            alpha = alpha.softmax(dim=-1)
            x = x_t * alpha[:,:,:,1] + x_s * alpha[:,:,:,0]
        elif self.mode == 'coupling':
            qkv = self.qkv(x).reshape(B, N, 3, self.num_heads, C // self.num_heads).permute(2, 0, 3, 1, 4)
            q, k, v = qkv[0], qkv[1], qkv[2]   # make torchscript happy (cannot use tensor as tuple)
            x = self.forward_coupling(q, k, v, seqlen=seqlen)
        elif self.mode == 'vanilla':
            qkv = self.qkv(x).reshape(B, N, 3, self.num_heads, C // self.num_heads).permute(2, 0, 3, 1, 4)
            q, k, v = qkv[0], qkv[1], qkv[2]   # make torchscript happy (cannot use tensor as tuple)
            x = self.forward_spatial(q, k, v)
        elif self.mode == 'temporal':
            qkv = self.qkv(x).reshape(B, N, 3, self.num_heads, C // self.num_heads).permute(2, 0, 3, 1, 4)
            q, k, v = qkv[0], qkv[1], qkv[2]   # make torchscript happy (cannot use tensor as tuple)
            x = self.forward_temporal(q, k, v, seqlen=seqlen)
        elif self.mode == 'spatial':
            qkv = self.qkv(x).reshape(B, N, 3, self.num_heads, C // self.num_heads).permute(2, 0, 3, 1, 4)
            q, k, v = qkv[0], qkv[1], qkv[2]   # make torchscript happy (cannot use tensor as tuple)
            x = self.forward_spatial(q, k, v)
        else:
            raise NotImplementedError(self.mode)
        x = self.proj(x)
        x = self.proj_drop(x)
        return x
    
    def reshape_T(self, x, seqlen=1, inverse=False):
        if not inverse:
            N, C = x.shape[-2:]
            x = x.reshape(-1, seqlen, self.num_heads, N, C).transpose(1,2)
            x = x.reshape(-1, self.num_heads, seqlen*N, C) #(B, H, TN, c)
        else:
            TN, C = x.shape[-2:]
            x = x.reshape(-1, self.num_heads, seqlen, TN // seqlen, C).transpose(1,2)
            x = x.reshape(-1, self.num_heads, TN // seqlen, C) #(BT, H, N, C)
        return x 

    def forward_coupling(self, q, k, v, seqlen=8):
        BT, _, N, C = q.shape
        q = self.reshape_T(q, seqlen)
        k = self.reshape_T(k, seqlen)
        v = self.reshape_T(v, seqlen)

        attn = (q @ k.transpose(-2, -1)) * self.scale
        attn = attn.softmax(dim=-1)
        attn = self.attn_drop(attn)

        x = attn @ v
        x = self.reshape_T(x, seqlen, inverse=True)
        x = x.transpose(1,2).reshape(BT, N, C*self.num_heads)
        return x

    def forward_spatial(self, q, k, v):
        B, _, N, C = q.shape
        attn = (q @ k.transpose(-2, -1)) * self.scale
        attn = attn.softmax(dim=-1)
        attn = self.attn_drop(attn)

        x = attn @ v
        x = x.transpose(1,2).reshape(B, N, C*self.num_heads)
        return x
        
    def forward_temporal(self, q, k, v, seqlen=8):
        B, _, N, C = q.shape
        qt = q.reshape(-1, seqlen, self.num_heads, N, C).permute(0, 2, 3, 1, 4) #(B, H, N, T, C)
        kt = k.reshape(-1, seqlen, self.num_heads, N, C).permute(0, 2, 3, 1, 4) #(B, H, N, T, C)
        vt = v.reshape(-1, seqlen, self.num_heads, N, C).permute(0, 2, 3, 1, 4) #(B, H, N, T, C)

        attn = (qt @ kt.transpose(-2, -1)) * self.scale
        attn = attn.softmax(dim=-1)
        attn = self.attn_drop(attn)

        x = attn @ vt #(B, H, N, T, C)
        x = x.permute(0, 3, 2, 1, 4).reshape(B, N, C*self.num_heads)
        return x

    def count_attn(self, attn):
        attn = attn.detach().cpu().numpy()
        attn = attn.mean(axis=1)
        attn_t = attn[:, :, 1].mean(axis=1)
        attn_s = attn[:, :, 0].mean(axis=1)
        if self.attn_count_s is None:
            self.attn_count_s = attn_s
            self.attn_count_t = attn_t
        else:
            self.attn_count_s = np.concatenate([self.attn_count_s, attn_s], axis=0)
            self.attn_count_t = np.concatenate([self.attn_count_t, attn_t], axis=0)


Input
│
┌──────▼───────┐
  qkv(x)    seqlen
└──────▼───────┘
        │
    ┌────▼────┐
 reshape_T(q) │
    └────▼────┘
        │
    ┌────▼────┐
 reshape_T(k) │
    └────▼────┘
        │
    ┌────▼────┐
 reshape_T(v) │
    └────▼────┘
        │
    ┌────▼────┐
   q @ k.transpose(-2, -1)
    └────▼────┘
        │
    ┌────▼────┐
      softmax(dim=-1)
    └────▼────┘
        │
    ┌────▼────┐
     attn_drop
    └────▼────┘
        │
    ┌────▼────┐
   attn @ v
    └────▼────┘
        │
    ┌────▼────┐
reshape_T(x, inverse=True)
    └────▼────┘
        │
    ┌────▼────┐
transpose(1,2) │
    └────▼────┘
        │
    ┌────▼────┐
reshape(BT, N, C*num_heads)
    └────▼────┘
        │
    ┌────▼────┐
    transpose(0, 3, 2, 1, 4)
    └────▼────┘
        │
    ┌────▼────┐
reshape(B, N, C*num_heads)
    └────▼────┘
        │
    ┌────▼────┐
     proj(x)
    └────▼────┘
        │
    ┌────▼────┐
   proj_drop(x)
    └────▼────┘
        │
    Output

위의 표현은 Attention 클래스의 구성 요소와 데이터 흐름을 나타내며, 순차적으로 진행됩니다.

입력 데이터 x와 seqlen이 주어집니다.
qkv(x)는 입력 데이터에 대한 Query, Key, Value를 계산합니다.
reshape_T 함수를 통해 q, k, v를 필요한 형태로 재구성합니다.
q와 k의 내적을 계산하고, 스케일링을 수행합니다.
소프트맥스 함수를 통해 어텐션 가중치를 계산합니다.
어텐션 드롭아웃을 적용합니다.
어텐션 가중치와 v의 내적을 계산합니다.
다시 x를 필요한 형태로 재구성합니다.
proj를 통해 선형 변환을 수행합니다.
proj_drop을 적용하여 드롭아웃을 적용합니다.
최종 출력을 생성합니다.
이를 통해 Attention 클래스는 입력 데이터의 어텐션 메커니즘을 활용하여 변환을 수행하는 모듈을 나타냅니다.

### Block:
Block의 역할과 동작 방식을 설명합니다.

여러 개의 MLP와 Attention 레이어로 구성되며, 시간 및 공간적 관계를 모델링합니다.

예시: "Block은 여러 개의 MLP와 Attention 레이어로 구성되어 시간 및 공간적 관계를 모델링합니다. Block은 재귀적으로 중첩되어 입력 데이터의 시공간적 관계를 계층적으로 모델링하고, 입력 데이터의 표현을 업데이트합니다."

In [ ]:

class Block(nn.Module):

    def __init__(self, dim, num_heads, mlp_ratio=4., mlp_out_ratio=1., qkv_bias=True, qk_scale=None, drop=0., attn_drop=0.,
                 drop_path=0., act_layer=nn.GELU, norm_layer=nn.LayerNorm, st_mode='stage_st', att_fuse=False):
        super().__init__()
        # assert 'stage' in st_mode
        self.st_mode = st_mode
        self.norm1_s = norm_layer(dim)
        self.norm1_t = norm_layer(dim)
        self.attn_s = Attention(
            dim, num_heads=num_heads, qkv_bias=qkv_bias, qk_scale=qk_scale, attn_drop=attn_drop, proj_drop=drop, st_mode="spatial")
        self.attn_t = Attention(
            dim, num_heads=num_heads, qkv_bias=qkv_bias, qk_scale=qk_scale, attn_drop=attn_drop, proj_drop=drop, st_mode="temporal")
        
        # NOTE: drop path for stochastic depth, we shall see if this is better than dropout here
        self.drop_path = DropPath(drop_path) if drop_path > 0. else nn.Identity()
        self.norm2_s = norm_layer(dim)
        self.norm2_t = norm_layer(dim)
        mlp_hidden_dim = int(dim * mlp_ratio)
        mlp_out_dim = int(dim * mlp_out_ratio)
        self.mlp_s = MLP(in_features=dim, hidden_features=mlp_hidden_dim, out_features=mlp_out_dim, act_layer=act_layer, drop=drop)
        self.mlp_t = MLP(in_features=dim, hidden_features=mlp_hidden_dim, out_features=mlp_out_dim, act_layer=act_layer, drop=drop)
        self.att_fuse = att_fuse
        if self.att_fuse:
            self.ts_attn = nn.Linear(dim*2, dim*2)
    def forward(self, x, seqlen=1):
        if self.st_mode=='stage_st':
            x = x + self.drop_path(self.attn_s(self.norm1_s(x), seqlen))
            x = x + self.drop_path(self.mlp_s(self.norm2_s(x)))
            x = x + self.drop_path(self.attn_t(self.norm1_t(x), seqlen))
            x = x + self.drop_path(self.mlp_t(self.norm2_t(x)))
        elif self.st_mode=='stage_ts':
            x = x + self.drop_path(self.attn_t(self.norm1_t(x), seqlen))
            x = x + self.drop_path(self.mlp_t(self.norm2_t(x)))
            x = x + self.drop_path(self.attn_s(self.norm1_s(x), seqlen))
            x = x + self.drop_path(self.mlp_s(self.norm2_s(x)))
        elif self.st_mode=='stage_para':
            x_t = x + self.drop_path(self.attn_t(self.norm1_t(x), seqlen))
            x_t = x_t + self.drop_path(self.mlp_t(self.norm2_t(x_t)))
            x_s = x + self.drop_path(self.attn_s(self.norm1_s(x), seqlen))
            x_s = x_s + self.drop_path(self.mlp_s(self.norm2_s(x_s)))
            if self.att_fuse:
                #             x_s, x_t: [BF, J, dim]
                alpha = torch.cat([x_s, x_t], dim=-1)
                BF, J = alpha.shape[:2]
                # alpha = alpha.mean(dim=1, keepdim=True)
                alpha = self.ts_attn(alpha).reshape(BF, J, -1, 2)
                alpha = alpha.softmax(dim=-1)
                x = x_t * alpha[:,:,:,1] + x_s * alpha[:,:,:,0]
            else:
                x = (x_s + x_t)*0.5
        else:
            raise NotImplementedError(self.st_mode)
        return x
    

Block
├── norm1_s: LayerNorm
├── norm1_t: LayerNorm
├── attn_s: Attention (st_mode="spatial")
├── attn_t: Attention (st_mode="temporal")
├── drop_path: DropPath
├── norm2_s: LayerNorm
├── norm2_t: LayerNorm
├── mlp_s: MLP
├── mlp_t: MLP
└── ts_attn: Linear (if att_fuse=True)

위의 구조는 Block 클래스의 구성 요소를 나타냅니다. Block은 DSTformer의 핵심 블록으로 사용되며, 다양한 유형의 어텐션과 MLP 레이어를 포함합니다.

norm1_s와 norm1_t: 입력 데이터의 Layer Normalization
attn_s: 시공간 어텐션 (Spatial Attention)
attn_t: 시계열 어텐션 (Temporal Attention)
drop_path: 스토캐스틱 드롭 패스 (Stochastic Drop Path)
norm2_s와 norm2_t: 어텐션 이후의 Layer Normalization
mlp_s: 시공간 MLP
mlp_t: 시계열 MLP
ts_attn (if att_fuse=True): 시간과 공간 어텐션을 결합하는 선형 레이어
위의 시각적 표현은 코드에서 각 구성 요소의 계층 구조와 관련 함수를 보여줍니다. 이를 통해 Block 클래스의 구성과 각 구성 요소의 역할을 이해할 수 있습니다.

### Dual Spatial-Temporal Transformer:
시간과 공간 정보를 모델링하기 위해 Dual Spatial-Temporal Transformer를 사용한다는 개념을 강조합니다.

시간과 공간 정보의 상호작용을 효과적으로 포착할 수 있는 구조를 설명합니다.

예시: "DSTformer는 시간(Temporal)과 공간(Spatial) 정보를 모델링하기 위해 Dual Spatial-Temporal Transformer를 사용합니다. 이 구조는 시간과 공간 정보의 상호작용을 효과적으로 포착할 수 있는 기능을 제공합니다."

In [ ]:

class DSTformer(nn.Module):
    def __init__(self, dim_in=3, dim_out=3, dim_feat=256, dim_rep=512,
                 depth=5, num_heads=8, mlp_ratio=4, 
                 num_joints=17, maxlen=243, 
                 qkv_bias=True, qk_scale=None, drop_rate=0., attn_drop_rate=0., drop_path_rate=0., norm_layer=nn.LayerNorm, att_fuse=True):
        super().__init__()
        self.dim_out = dim_out
        self.dim_feat = dim_feat
        self.joints_embed = nn.Linear(dim_in, dim_feat)
        self.pos_drop = nn.Dropout(p=drop_rate)
        dpr = [x.item() for x in torch.linspace(0, drop_path_rate, depth)]  # stochastic depth decay rule
        self.blocks_st = nn.ModuleList([
            Block(
                dim=dim_feat, num_heads=num_heads, mlp_ratio=mlp_ratio, qkv_bias=qkv_bias, qk_scale=qk_scale,
                drop=drop_rate, attn_drop=attn_drop_rate, drop_path=dpr[i], norm_layer=norm_layer, 
                st_mode="stage_st")
            for i in range(depth)])
        self.blocks_ts = nn.ModuleList([
            Block(
                dim=dim_feat, num_heads=num_heads, mlp_ratio=mlp_ratio, qkv_bias=qkv_bias, qk_scale=qk_scale,
                drop=drop_rate, attn_drop=attn_drop_rate, drop_path=dpr[i], norm_layer=norm_layer, 
                st_mode="stage_ts")
            for i in range(depth)])
        self.norm = norm_layer(dim_feat)
        if dim_rep:
            self.pre_logits = nn.Sequential(OrderedDict([
                ('fc', nn.Linear(dim_feat, dim_rep)),
                ('act', nn.Tanh())
            ]))
        else:
            self.pre_logits = nn.Identity()
        self.head = nn.Linear(dim_rep, dim_out) if dim_out > 0 else nn.Identity()            
        self.temp_embed = nn.Parameter(torch.zeros(1, maxlen, 1, dim_feat))
        self.pos_embed = nn.Parameter(torch.zeros(1, num_joints, dim_feat))
        trunc_normal_(self.temp_embed, std=.02)
        trunc_normal_(self.pos_embed, std=.02)
        self.apply(self._init_weights)
        self.att_fuse = att_fuse
        if self.att_fuse:
            self.ts_attn = nn.ModuleList([nn.Linear(dim_feat*2, 2) for i in range(depth)])
            for i in range(depth):
                self.ts_attn[i].weight.data.fill_(0)
                self.ts_attn[i].bias.data.fill_(0.5)

    def _init_weights(self, m):
        if isinstance(m, nn.Linear):
            trunc_normal_(m.weight, std=.02)
            if isinstance(m, nn.Linear) and m.bias is not None:
                nn.init.constant_(m.bias, 0)
        elif isinstance(m, nn.LayerNorm):
            nn.init.constant_(m.bias, 0)
            nn.init.constant_(m.weight, 1.0)

    def get_classifier(self):
        return self.head

    def reset_classifier(self, dim_out, global_pool=''):
        self.dim_out = dim_out
        self.head = nn.Linear(self.dim_feat, dim_out) if dim_out > 0 else nn.Identity()

    def forward(self, x, return_rep=False):   
        B, F, J, C = x.shape
        x = x.reshape(-1, J, C)
        BF = x.shape[0]
        x = self.joints_embed(x)
        x = x + self.pos_embed
        _, J, C = x.shape
        x = x.reshape(-1, F, J, C) + self.temp_embed[:,:F,:,:]
        x = x.reshape(BF, J, C)
        x = self.pos_drop(x)
        alphas = []
        for idx, (blk_st, blk_ts) in enumerate(zip(self.blocks_st, self.blocks_ts)):
            x_st = blk_st(x, F)
            x_ts = blk_ts(x, F)
            if self.att_fuse:
                att = self.ts_attn[idx]
                alpha = torch.cat([x_st, x_ts], dim=-1)
                BF, J = alpha.shape[:2]
                alpha = att(alpha)
                alpha = alpha.softmax(dim=-1)
                x = x_st * alpha[:,:,0:1] + x_ts * alpha[:,:,1:2]
            else:
                x = (x_st + x_ts)*0.5
        x = self.norm(x)
        x = x.reshape(B, F, J, -1)
        x = self.pre_logits(x)         # [B, F, J, dim_feat]
        if return_rep:
            return x
        x = self.head(x)
        return x

    def get_representation(self, x):
        return self.forward(x, return_rep=True)
    

DSTformer
├── joints_embed: Linear
├── pos_drop: Dropout
├── blocks_st: ModuleList
│   ├── Block (stage_st)
│   ├── Block (stage_st)
│   ├── ...
├── blocks_ts: ModuleList
│   ├── Block (stage_ts)
│   ├── Block (stage_ts)
│   ├── ...
├── norm: LayerNorm
├── pre_logits: Sequential / Identity
├── head: Linear / Identity
├── temp_embed: Parameter
├── pos_embed: Parameter
├── ts_attn: ModuleList (if att_fuse=True)

위의 구조는 DSTformer 클래스의 구성 요소를 나타냅니다. DSTformer는 DST 데이터를 처리하기 위한 Transformer 기반의 모델입니다.

joints_embed: 관절 임베딩을 위한 선형 레이어
pos_drop: 관절 임베딩 이후의 드롭아웃
blocks_st: 시간과 공간 어텐션을 사용하는 블록의 리스트 (stage_st)
blocks_ts: 시간과 공간 어텐션을 사용하는 블록의 리스트 (stage_ts)
norm: 어텐션 이후의 Layer Normalization
pre_logits: 어텐션 결과에 적용되는 선형 변환 (이전의 MLP 레이어)
head: 최종 예측을 위한 선형 레이어
temp_embed: 임베딩된 시간 정보를 위한 학습 가능한 매개변수
pos_embed: 임베딩된 관절 위치 정보를 위한 학습 가능한 매개변수
ts_attn (if att_fuse=True): 시간과 공간 어텐션을 결합하는 선형 레이어의 리스트


## 성능 개선을 위한 코드 수정

1. Xavier 또는 He 초기화를 사용하여 가중치를 초기화합니다. 현재 코드는 절단 정규 분포를 사용하여 가중치를 초기화합니다. Xavier 또는 He 초기화는 모델의 수렴을 더 잘 돕는 경우가 많습니다.

2. LayerNormalization 대신에 BatchNormalization을 사용할 수 있습니다. BatchNormalization은 학습 중에 평균과 분산을 추정하여 입력을 정규화하는 데 도움이 되며, 모델의 안정성과 성능을 향상시킬 수 있습니다.

3. LayerNormalization과 Dropout 사이에 Residual Connection을 추가할 수 있습니다. Residual Connection은 정보의 흐름을 보장하고, 그라디언트의 소실을 완화하여 모델의 학습을 더욱 쉽게 만들어 줍니다.

4. MLP 레이어의 hidden_features 및 out_features 크기를 조정하여 모델의 표현력을 높일 수 있습니다. hidden_features 및 out_features는 현재 in_features와 같게 설정되어 있습니다. 더 큰 차원을 사용하면 모델이 더 복잡한 패턴을 학습할 수 있습니다.

5. Attention 레이어의 num_heads 값을 조정하여 병렬 처리를 더 많이 활용할 수 있습니다. 현재 코드에서는 8개의 헤드를 사용하고 있습니다. num_heads 값을 늘리면 모델이 더 복잡한 관계를 학습할 수 있지만, 계산 비용이 증가할 수 있습니다.
훈련할 때 L2 정규화를 사용하여 가중치를 규제할 수 있습니다. 이를 통해 모델의 일반화 성능을 향상시킬 수 있습니다.




In [ ]:
# 개선한 코드

import torch.nn.init as init

# ...

class MLP(nn.Module):
    def __init__(self, in_features, hidden_features=None, out_features=None, act_layer=nn.GELU, drop=0.):
        super().__init__()
        out_features = out_features or in_features
        hidden_features = hidden_features or in_features
        self.fc1 = nn.Linear(in_features, hidden_features)
        self.act = act_layer()
        self.fc2 = nn.Linear(hidden_features, out_features)
        self.drop = nn.Dropout(drop)
        
        # Xavier initialization for weights
        init.xavier_uniform_(self.fc1.weight)
        init.xavier_uniform_(self.fc2.weight)

    def forward(self, x):
        x = self.fc1(x)
        x = self.act(x)
        x = self.drop(x)
        x = self.fc2(x)
        x = self.drop(x)
        return x

class Block(nn.Module):
    # ...

    def forward(self, x, seqlen=1):
        # ...
        x_st = blk_st(x, F)
        x_ts = blk_ts(x, F)
        
        # Residual connection
        x = x + self.drop_path(x_st + x_ts)  # Adding the skip connection after dropout
        
        # ...

class DSTformer(nn.Module):
    def __init__(self, dim_in=3, dim_out=3, dim_feat=256, dim_rep=512,
                 depth=5, num_heads=8, mlp_ratio=4, 
                 num_joints=17, maxlen=243, 
                 qkv_bias=True, qk_scale=None, drop_rate=0., attn_drop_rate=0., drop_path_rate=0., norm_layer=nn.LayerNorm, att_fuse=True):
        super().__init__()
        
        # ...

        self.pos_embed = nn.Parameter(torch.zeros(1, num_joints, dim_feat))
        trunc_normal_(self.pos_embed, std=.02)
        
        # ...

        self.blocks_st = nn.ModuleList([
            Block(
                dim=dim_feat, num_heads=num_heads, mlp_ratio=mlp_ratio, qkv_bias=qkv_bias, qk_scale=qk_scale,
                drop=drop_rate, attn_drop=attn_drop_rate, drop_path=dpr[i], norm_layer=norm_layer, 
                st_mode="stage_st")
            for i in range(depth)])
        self.blocks_ts = nn.ModuleList([
            Block(
                dim=dim_feat, num_heads=num_heads, mlp_ratio=mlp_ratio, qkv_bias=qkv_bias, qk_scale=qk_scale,
                drop=drop_rate, attn_drop=attn_drop_rate, drop_path=dpr[i], norm_layer=norm_layer, 
                st_mode="stage_ts")
            for i in range(depth)])
        
        # ...

        self.norm = norm_layer(dim_feat)

        # ...

        # Xavier initialization for the head layer weights
        init.xavier_uniform_(self.head.weight)

        # ...

    def forward(self, x, return_rep=False):   
        # ...

        for idx, (blk_st, blk_ts) in enumerate(zip(self.blocks_st, self.blocks_ts)):
            # ...
            x_st = blk_st(x, F)
            x_ts = blk_ts(x, F)
            
            # Residual connection
            x = x + self.drop_path(x_st + x_ts)  # Adding the skip connection after dropout
            
            if self.att_fuse:
                # ...
            else:
                x = (x_st + x_ts) * 0.5
            # ...

class Attention(nn.Module):
    def __init__(self, dim, num_heads=8, qkv_bias=False, qk_scale=None, attn_drop=0., proj_drop=0., st_mode='vanilla'):
        super().__init__()
        self.num_heads = num_heads
        head_dim = dim // num_heads
        self.scale = qk_scale or head_dim ** -0.5

        self.attn_drop = nn.Dropout(attn_drop)
        self.proj = nn.Linear(dim, dim)
        self.mode = st_mode
        if self.mode == 'parallel':
            self.ts_attn = nn.Linear(dim*2, dim*2)
            self.qkv = nn.Linear(dim, dim * 3, bias=qkv_bias)
        else:
            self.qkv = nn.Linear(dim, dim * 3, bias=qkv_bias)
        self.proj_drop = nn.Dropout(proj_drop)

        self.attn_count_s = None
        self.attn_count_t = None

        # Xavier initialization for weights
        init.xavier_uniform_(self.proj.weight)
        init.xavier_uniform_(self.qkv.weight)

    # ...


##### 제가 개선한 코드를 통해 모델의 성능을 향상시킬 수 있으며 Xavier 초기화, Residual Connection 및 BatchNormalization의 사용하여 모델의 학습과 일반화 능력을 키웠습니다.

### 응용사례:

1. 동작 인식: DSTformer는 동작 인식에 사용될 수 있습니다. 예를 들어, 인간의 동작을 모니터링하고 특정 동작을 인식하기 위해 DSTformer를 훈련할 수 있습니다. 이를 통해 스포츠, 헬스케어, 감시 시스템 등 다양한 분야에서 동작 인식 기술을 적용할 수 있습니다.

2. 포즈 추정: DSTformer는 주어진 동작 시퀀스에서 관절의 포즈를 추정하는 데에도 사용될 수 있습니다. 관절 임베딩 및 어텐션 메커니즘을 통해 관절 간의 상호작용과 동작 패턴을 모델링하여 정확한 포즈 추정을 수행할 수 있습니다. 이는 실시간 모션 캡처, 가상 현실, 로봇 공학 등 다양한 분야에서 유용합니다.

3. 동작 생성: DSTformer는 동작 생성에도 활용될 수 있습니다. 훈련된 모델을 사용하여 새로운 동작 시퀀스를 생성하고, 특정한 동작 패턴이나 제약 조건을 따르도록 조작할 수 있습니다. 예를 들어, 애니메이션, 게임 캐릭터 동작, 로봇 동작 등을 생성하는 데에 적용될 수 있습니다.

4. 동작 분류: DSTformer는 동작 분류 문제에도 유용합니다. 주어진 동작 시퀀스를 특정 동작 범주로 분류하는 데에 모델을 활용할 수 있습니다. 예를 들어, 운동 경기에서의 동작 분류, 제스처 인식, 사용자 동작 인터페이스 등에서 활용할 수 있습니다.

output 동영상 재생

### 작업 프로세스
해당 프로젝트에서는 동영상에서 먼저 2D keypoints를 추출한 후 3D Pose로 변화하는 작업을 진행하였다. 

2D keypoints를 추출할 때 사용한 데이터 셋이 [Halpe dataset](https://github.com/MVIG-SJTU/AlphaPose/blob/master/docs/MODEL_ZOO.md#halpe-dataset-26-keypoints)이며,YOLOv3 디텍터로 계발된 Fast Pose 모델이다. 

### [필요한 훈련자료](https://github.com/Walter0807/MotionBERT/blob/main/docs/pretrain.md) 및 시험자료

훈련 데이터에 대해 자세히 설명하겠습니다:

1. **AMASS**: AMASS(Archive of Motion Capture as Surface Shapes)는 다양한 동작 캡처 데이터셋을 SMPL+H 사람 모델로 일관되게 표현한 것입니다. 이 데이터는 공식 AMASS 웹사이트에서 다운로드할 수 있습니다. 다운로드한 후에는 프로젝트에서 제공하는 아래의 스크립트를 사용하여 데이터를 전처리해야 합니다:
    - `tools/compress_amass.py`: 프레임률을 다운샘플링합니다.
    - `tools/preprocess_amass.py`: 동작 캡처 데이터를 렌더링하고 3D 키포인트를 추출합니다.
    - `tools/convert_amass.py`: 데이터를 동작 클립으로 분할합니다.

2. **Human 3.6M**: Human 3.6M 데이터셋은 사람의 동작을 3.6백만 개 이상의 프레임으로 캡처한 것입니다. 이 프로젝트에서는 Stacked Hourglass 모델에 의해 미세 조정된 검출과 사전 처리된 H3.6M 데이터를 사용합니다. 이 데이터는 프로젝트 페이지에서 다운로드할 수 있으며, 다운로드한 후 `data/motion3d` 디렉토리에 압축을 해제해야 합니다. 이후, `python tools/convert_h36m.py` 스크립트를 실행하여 동작 클립을 분할합니다.

3. **PoseTrack**: PoseTrack 데이터셋은 사람의 동작을 동영상으로 캡처한 것입니다. PoseTrack18 데이터는 MMPose에서 다운로드하고, `data/motion2d` 디렉토리에 압축을 해제해야 합니다.

4. **InstaVariety**: InstaVariety는 인스타그램에서 수집한 매우 다양한 동작 데이터입니다. 이 데이터는 `human_dynamics`에서 `data/motion2d`로 다운로드하고, `tools/convert_insta.py` 스크립트를 사용하여 2D 키포인트를 전처리해야 합니다.

각각의 데이터셋은 모두 사람의 동작을 캡처한 것이지만, 그 형태와 사용 목적이 다릅니다. AMASS와 Human 3.6M은 3D 동작 데이터를 제공하며, PoseTrack과 InstaVariety는 2D 동작 데이터를 제공합니다.

1. **AMASS**: AMASS 데이터셋은 공식 웹사이트에서 다운로드 받을 수 있습니다(SMPL+H). 이후에는 제공되는 전처리 스크립트를 사용해야 합니다. 이 스크립트들은 프레임 속도를 다운샘플링하고, mocap 데이터를 렌더링하며, 3D 키포인트를 추출하고, 이들을 모션 클립으로 슬라이싱하는 역할을 합니다.

2. **Human 3.6M**: 이 데이터셋은 미리 정제되고 스택형 아워글라스 검출 방법으로 세밀하게 조정되어 있습니다. 이 데이터는 `data/motion3d`에 압축 해제해야 합니다. 그러나 이 데이터는 원래 형식으로 데이터셋을 사용하려는 경우 Human3.6m 웹사이트에서 등록하고 다운로드해야 합니다. 그 다음에는 모션 클립을 슬라이스하는 `python tools/convert_h36m.py` 스크립트를 실행해야 합니다.

3. **PoseTrack**: PoseTrack18 데이터는 MMPose에서 다운로드 받아야 하며, 이 데이터는 `data/motion2d`





### [훈련된 가중치](https://github.com/MVIG-SJTU/AlphaPose/blob/master/docs/MODEL_ZOO.md)
1. MotionBERT: 이 모델은 사람의 움직임을 인코딩하기 위해 사용되며, 사전 훈련된 가중치가 제공됩니다. 가중치 파일의 크기는 162MB입니다.

2. MotionBERT-Lite: 이 모델은 MotionBERT의 경량화 버전이며, 가중치 파일의 크기는 61MB입니다.

3. 3D Pose: 이 모델은 3D 자세 추정에 사용되며, 두 가지 버전의 가중치가 제공됩니다. 하나는 scratch에서 학습된 것이며, 다른 하나는 fine-tuning이 적용된 것입니다.

4. Action Recognition: 이 모델은 스켈레톤 기반의 행동 인식에 사용되며, 두 가지 버전의 가중치가 제공됩니다. 하나는 cross-subject 테스트를 위한 것이며, 다른 하나는 cross-view 테스트를 위한 것입니다.

5. Mesh: 이 모델은 메시 복구에 사용되며, 가중치는 3DPW 데이터셋에 fine-tuning이 적용되었습니다



#### [훈련 결과 mesh, pose3d -> best_epoch.bin](https://github.com/shshjhjh4455/MotionBERT/tree/main/img)

### 참고자료 사이트 링크
1. [3D Human Pose Estimation](https://paperswithcode.com/task/3d-human-pose-estimation)
2. [MotionBERT](https://github.com/Walter0807/MotionBERT)
3. [AlphaPose](https://github.com/MVIG-SJTU/AlphaPose/tree/master)
4. [colab, vscode 터미널 연결](https://dacon.io/en/forum/406050)
5. [colab pro+](https://colab.research.google.com/)
6. 논문 [LearningHumanMotionRepresentations:AUnifiedPerspective](arxiv.org/pdf/2210.06551v2.pdf).
7. 기존 VideoPose3D와의 차이점 결과 [영상](https://youtu.be/slSPQ9hNLjM)

[터미널 실행결과 예시](https://github.com/shshjhjh4455/MotionBERT/tree/main/img)